In [1]:
import pandas as pd
import pandahouse as ph

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. 
Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 заданий.

## Задание 1
Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов

In [2]:
# подключение к базе default
connection_default = {'host' : 'http://clickhouse.beslan.pro:8080',
                      'database'  : 'default',
                      'user' :'student', 
                      'password' : 'dpo_python_2020'
                     }

In [3]:
q1 = '''
SELECT month_date,
       count(DISTINCT st_id) AS quantity_student
FROM (
      SELECT toStartOfMonth(timest) AS month_date,
             st_id
      FROM default.peas
      WHERE correct = 1
      GROUP BY month_date, st_id
      HAVING count(st_id)>=20
     )
GROUP BY month_date
   '''
# отправляем запрос и записываем результат в пандасовский датафрейм
q_test1 = ph.read_clickhouse(query=q1, connection=connection_default)
q_test1

,month_date,quantity_student
0,2021-10-01,136


## Задание 2
Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:   
ARPU   
ARPAU   
CR в покупку   
СR активного пользователя в покупку   
CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

In [4]:
q2 =  '''
SELECT f.test_grp,
       ROUND(sum_all_money/cnt_all, 2) ARPU,
       ROUND(sum_active_money/cnt_active_all, 2) ARPAU,
       ROUND(cnt_all_payed*100/cnt_all, 2) CR,
       ROUND(cnt_all_payed_active*100/cnt_active_all, 2) CR_active,
       ROUND(cnt_all_payed_active_math*100/cnt_active_math, 2) CR_active_math
FROM
    (
    SELECT s.test_grp AS test_grp,
           COUNT(*) cnt_all,
           COUNT(IF(c.check=1, 1, null)) cnt_all_payed,
           COUNT(IF(c.check=1, p.cnt_active, null)) cnt_all_payed_active,
           COUNT(IF(c.check=1, p.cnt_active_math, null)) cnt_all_payed_active_math,
           SUM(c.sum_money) sum_all_money,
           SUM(IF(p.cnt_active=1,c.sum_money,0)) sum_active_money,
           SUM(c.cnt_math_payed) cnt_math_payed,
           SUM(c.sum_math_payed) sum_math_money,
           SUM(p.cnt_active) cnt_active_all,
           SUM(p.cnt_active_math) cnt_active_math
    FROM studs s
    LEFT JOIN --присоединяем информацию о покупках
            (
            SELECT c.st_id,
                   1 check, -- поле отражает факт покупки (1 - была покупка)
                   SUM(c.money) sum_money,
                   MAX(IF(c.subject = 'Math', 1, 0)) cnt_math_payed,
                   SUM(IF(c.subject = 'Math', c.money, 0)) sum_math_payed
                  
            FROM final_project_check c
            GROUP BY c.st_id
            ) c
    ON (c.st_id = s.st_id)
    LEFT JOIN  -- присоединяем информацию об активных пользователях
            (
            SELECT p.st_id,
                   1 check, -- поле отражает факт покупки (1 - была покупка)
                   IF(COUNT(*)>10,1,NULL) cnt_active,             
                   
                   IF(COUNT(IF(p.subject = 'Math', 1, NULL)) >= 2, 1, NULL) cnt_active_math
            FROM peas p
            WHERE p.correct = 1
            GROUP BY p.st_id
            ) p 
    ON (p.st_id = s.st_id)
    GROUP BY s.test_grp
    ) f
'''
q_test2 = ph.read_clickhouse(query=q2, connection=connection_default)
q_test2

,test_grp,ARPU,ARPAU,CR,CR_active,CR_active_math
0,control,4540.98,10393.70,4.92,11.02,10.20
1,pilot,11508.47,29739.58,10.85,26.04,14.29
